# Data analysis can be split into three elementary steps:
## 1. Importing the data
All the methods necessary to **read** the file contents, parse data and output it in desired format. Ie. something like that:
 
```
import pathlib
import pandas as pd

def data_import(self, path: pathlib.Path) -> pd.DataFrame:
    # ... 
    # dataframe =
    return dataframe
```

## 2. Processing the data
All the smoothing, normalization, interpolation, etc. should happen here. Implemented using composite design pattern.

## 3. Showing the data
ble ble

# Implementation using bridge design pattern/dependency injection:


In [22]:
import pathlib
from pathlib import Path
from IPython.display import display
from abc import ABC, abstractmethod
from typing import Callable, Optional, List
import pandas as pd
from pandas.errors import ParserError
import numpy as np
import itertools

class Handler(ABC):
    @abstractmethod
    def handle(self, *args, **kwargs):
        raise NotImplementedError("You should implement this!")
        
    def update_handler(self, new_handler: Callable) -> None:
        self.handle = new_handler
        
class FileHandler(Handler):
    def handle(self, path: pathlib.Path) -> pd.DataFrame:
        with open(path) as f:
            for i, row in enumerate(f):
                for delimiter, decimal in (itertools.product([';',',','\t'], ['.',','])):
                    try:
                        df = pd.read_csv(path ,delimiter=delimiter, decimal=decimal, index_col=0, skiprows=i,header=None, on_bad_lines='skip', dtype=np.float64)
                        

                        return df
                        # return df[df.columns[-1]]

                    except Exception:
                        pass
    
class Hitachi(Handler):
    def handle(self, path: pathlib.Path) -> pd.DataFrame:
        return pd.read_csv(path ,delimiter='\t',header=27,on_bad_lines='skip')

class Avantes(Handler):
    def handle(self, path: pathlib.Path) -> pd.DataFrame:
        return pd.read_csv(path,delimiter=';',decimal=',',header=5,on_bad_lines='skip')

class DataHandler(Handler):
    def __init__(self, handler: Optional[Callable]=None):
        super().__init__()
        
        if handler is not None:
            self.update_handler(handler)
        
    def handle(self,  data: pd.DataFrame) -> pd.DataFrame:
        return data

class Composite(DataHandler):
    def __init__(self) -> None:
        self.operations: List[DataHandler] = []
        
    def handle(self,  data: pd.DataFrame) -> pd.DataFrame:
        for o in self.operations:
            data = o.handle( data )
        return data

    def add(self, *operations: DataHandler) -> None:
        for o in operations:
            self.operations.append(o)

    def remove(self,*operations: DataHandler) -> None:
        for o in operations:
            self.operations.remove(o)
            
class DataModel(ABC):
    def __init__(self, path: pathlib.Path, reader: Optional[FileHandler]=FileHandler(), pipeline: Optional[DataHandler]=Composite()) -> None:
        self._reader=reader
        self._pipeline=pipeline
        self.path=path
    @property
    def path(self):
        return self._path
    
    @path.setter
    def path(self, path: pathlib.Path):
        self._path = path
    @property
    def raw_data(self):
        try:
            return self._reader.handle(self.path)
        except ParserError as e:
            display(repr(e))
            return pd.DataFrame()
    @property
    def data(self):
        return self._pipeline.handle(self.raw_data)
    
        
dm = DataModel(path=Path('../patka-pomiary/CPE45_PFOBPy_comocat.txt', reader=Hitachi()))
display(dm.data[:5])
dm._pipeline.add(DataHandler(handler=lambda df: df+5))
dm._pipeline.add(DataHandler(handler=lambda df: df+5))
dm._pipeline.add(DataHandler(handler=lambda df: df+5))
display(dm.data[:5])

dm2 = DataModel(path=Path('../patka-pomiary/func39.txt', reader=Avantes()))
display(dm2.data)



0
1100.0    0.0420
1099.0    0.0420
1098.0    0.0419
1097.0    0.0418
1096.0    0.0417
           ...  
404.0     0.0988
403.0     0.1040
402.0     0.1092
401.0     0.1146
400.0     0.1186
Name: 1, Length: 701, dtype: float64

0
1100.0    15.0420
1099.0    15.0420
1098.0    15.0419
1097.0    15.0418
1096.0    15.0417
           ...   
404.0     15.0988
403.0     15.1040
402.0     15.1092
401.0     15.1146
400.0     15.1186
Name: 1, Length: 701, dtype: float64

0
900.12      15.00000
903.33     330.33333
906.55     192.53333
909.76     258.40000
912.97     257.80000
             ...    
1692.65    103.06667
1695.73    125.40000
1698.80    131.60000
1701.87    179.33333
1704.94    140.00000
Name: 4, Length: 256, dtype: float64

In [23]:
df=dm.data

df[df.columns[-1]]

AttributeError: 'Series' object has no attribute 'columns'

In [ ]:
import pandas as pd

pd.show_versions()


INSTALLED VERSIONS
------------------
commit           : 5f648bf1706dd75a9ca0d29f26eadfbb595fe52b
python           : 3.9.7.final.0
python-bits      : 64
OS               : Windows
OS-release       : 10
Version          : 10.0.19042
machine          : AMD64
processor        : AMD64 Family 23 Model 24 Stepping 1, AuthenticAMD
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : English_Europe.1252

pandas           : 1.3.2
numpy            : 1.21.2
pytz             : 2021.1
dateutil         : 2.8.2
pip              : 22.0.3
setuptools       : 57.4.0
Cython           : None
pytest           : None
hypothesis       : None
sphinx           : None
blosc            : None
feather          : None
xlsxwriter       : None
lxml.etree       : 4.6.4
html5lib         : None
pymysql          : None
psycopg2         : None
jinja2           : 3.0.1
IPython          : 7.27.0
pandas_datareader: None
bs4              : 4.9.3
bottleneck       : None
fsspec          